In [1]:
from web_parser import AnswerParser, QuestionParser
import multiprocessing
from multiprocessing import Queue, Process
from selenium import webdriver
from pathlib import Path
import os
from util.repo import driver_root, database_root, drivers, git_repo_test
from data_handler import Master

In [2]:
q_id = 625769474
# q_id = 460106396
q_id = 327003589
# q_id = 459470265
q_list = [q_id]
config = {
    "simutaneously_scrap_and_store": False,
    "nr_answer_parsers": 2,
    "commiter_name":"test",
    "commiter_email":"test@mal.com"
}
db_path = os.path.join(database_root, "test.2.db")

# TODO
 - [x] test scrap then store
 - [ ] use event to communicate
 - [ ] test scrap and store at same time
 - [x] test on a git repo
 - [ ] test to create git repo and make a commit<br>
 https://stackoverflow.com/questions/71947784/what-does-webdriverwaitdriver-20-mean<br>
https://stackoverflow.com/questions/26869200/python-queue-module-get-stuck

---
 - [ ] scrap author
 - [ ] init git repo (partly done)
 - [ ] init db (partly done)
 - [ ] merge repo
 - [ ] revert git operation
 - [ ] scroll
 - [x] git message
 - [ ] configurable xpath rules


In [4]:
q_task = Queue()
q_result = Queue()

In [6]:
class Controller:
    def __init__(
        self, 
    ):
        self.processes = []
        pass

In [7]:
question_driver = webdriver.Chrome(drivers["chrome"])
question_parser = QuestionParser(
    queue_put_result=q_result,
    queue_put_task=q_task,
    driver=question_driver,
    logger=Logger("q")
)
answer_drivers = []
answer_parsers = []
for i in range(config["nr_answer_scapers"]):
    answer_drivers.append(
        webdriver.Chrome(drivers["chrome"])
    )
    answer_parsers.append(
        AnswerParser(
            queue_put_result=q_result,
            queue_get_task=q_task,
            driver=answer_drivers[i],
            logger=Logger("a_{}".format(i))
        )
    )
master = Master(
    db_path=db_path,
    git_repo_path=git_repo_test
)


In [8]:
if not config["simutaneously_scrap_and_store"]:
    print("scrap, then store")
    targets = [
        # target func and args
        (question_parser.start_parsing_list, (q_list,))
    ] + [
        (answer_parser.start_parsing, [])
        for answer_parser in answer_parsers
    ] + [
        (master.start, (config["commiter_name"],config["commiter_email"],q_result))
    ]
        
    
    processes = [
        Process(target=target_func, args=args)
        for target_func, args in targets
    ]
    
   
    
        
        
else:
    print(1)

scrap, then store


In [9]:
for p in processes[:-1]:
    p.start()

327003589

q:parsing question with id=327003589a_0:parsing answer with id=3255789284
a_1:parsing answer with id=701487690
q:parsed 327003589
q:fin
a_1:parsing answer with id=728565226
a_0:parsing answer with id=1890558990
a_0:parsing answer with id=722240235
a_1:stop
a_1:put
a_1:out
a_0:stop
a_0:put
a_0:out


In [10]:
processes[-1].start()
for p in processes:
    p.join()

In [10]:
# type(l)

search:process queue self._wait_for_tstate_lock() 